# Infrared Thermography (IRT) Reconstruction Demo
This notebook demonstrates a basic reconstruction pipeline for Infrared Thermography. It aims to reconstruct a subsurface property map (e.g., defects, heat sources) from surface temperature measurements over time. 
The `InfraredThermographyOperator` used here is a placeholder that simulates heat diffusion with a simple kernel. For realistic results, this operator would need to implement a more accurate heat equation solver.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Adjust path to import from reconlib 
import sys
# sys.path.append('../../../') # Adjust as needed

from reconlib.modalities.infrared_thermography.operators import InfraredThermographyOperator
from reconlib.modalities.infrared_thermography.reconstructors import tv_reconstruction_irt
from reconlib.modalities.infrared_thermography.utils import generate_irt_phantom, plot_irt_results

print(f"PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Setup Parameters and Phantom
We'll define parameters for a 2D subsurface map reconstruction.

In [ ]:
subsurface_image_shape = (64, 64)  # (Ny, Nx) for the subsurface property map
num_time_steps = 10                # Number of time frames for surface temperature

# Generate a phantom representing subsurface anomalies (e.g., defects or heat sources)
# 'rect_inclusion' simulates areas with different thermal resistance (phantom values around 1.0 normally, defects alter this)
# 'hotspot' simulates localized heat sources (phantom values are source intensities)
phantom_type = 'hotspot' # or 'rect_inclusion' 
true_subsurface_map = generate_irt_phantom(
    image_shape=subsurface_image_shape, 
    num_defects=3, 
    defect_type=phantom_type, 
    device=device
)

plt.figure(figsize=(6,6))
plt.imshow(true_subsurface_map.cpu().numpy(), cmap='hot')
plt.title(f'True Subsurface Map ({phantom_type})')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.colorbar(label='Property Value / Intensity')
plt.show()

## 2. Initialize Operator and Simulate Data
The IRT operator will simulate the surface temperature sequence based on the subsurface map.

In [ ]:
# The operator uses a default simple Gaussian blur kernel if none is provided.
irt_operator = InfraredThermographyOperator(
    image_shape=subsurface_image_shape,
    time_steps=num_time_steps,
    device=device
)

# Simulate surface temperature data using the forward operator
# Note: This uses the placeholder diffusion model!
y_surface_temps = irt_operator.op(true_subsurface_map)

print(f"Simulated surface temperature data shape: {y_surface_temps.shape}")

# Visualize a few time frames of the simulated surface temperature
fig, axes = plt.subplots(1, min(num_time_steps, 4), figsize=(min(num_time_steps,4)*4, 4))
if min(num_time_steps,4) == 1: axes = [axes] # Make iterable if single subplot
for i, ax_idx in enumerate(np.linspace(0, num_time_steps-1, min(num_time_steps, 4)).astype(int)):
    im = axes[i].imshow(y_surface_temps[ax_idx].cpu().numpy(), cmap='inferno')
    axes[i].set_title(f'Surface Temp (t={ax_idx})')
    axes[i].set_xlabel('X')
    axes[i].set_ylabel('Y')
    fig.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)
plt.suptitle('Simulated Surface Temperatures (Placeholder Model)')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## 3. Perform Reconstruction
Using Total Variation (TV) regularization with Proximal Gradient to reconstruct the subsurface map.

In [ ]:
lambda_tv_irt = 0.001     # TV regularization strength (may need tuning)
iterations_irt = 25       # Number of proximal gradient iterations (low for demo)
step_size_irt = 0.01      # Step size for proximal gradient
tv_prox_iters_irt = 5     # Iterations for TV prox

# Perform reconstruction
reconstructed_subsurface = tv_reconstruction_irt(
    y_surface_temperature_sequence=y_surface_temps,
    irt_operator=irt_operator,
    lambda_tv=lambda_tv_irt,
    iterations=iterations_irt,
    step_size=step_size_irt,
    tv_prox_iterations=tv_prox_iters_irt,
    # is_3d_tv is auto-detected from irt_operator.image_shape in the function
    verbose=True
)

print(f"Reconstructed subsurface map shape: {reconstructed_subsurface.shape}")

## 4. Display Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

im1 = axes[0].imshow(true_subsurface_map.cpu().numpy(), cmap='hot')
axes[0].set_title(f'Ground Truth Subsurface ({phantom_type})')
axes[0].set_xlabel('X (pixels)')
axes[0].set_ylabel('Y (pixels)')
fig.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)

im2 = axes[1].imshow(reconstructed_subsurface.cpu().numpy(), cmap='hot')
axes[1].set_title(f'Reconstructed Subsurface (TV, {iterations_irt} iters - Placeholder)')
axes[1].set_xlabel('X (pixels)')
axes[1].set_ylabel('Y (pixels)')
fig.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

# Using the utility plot function (currently a placeholder itself)
plot_irt_results(
    subsurface_map_true=true_subsurface_map,
    subsurface_map_recon=reconstructed_subsurface,
    surface_temp_sequence=y_surface_temps,
    time_slice_to_display=num_time_steps-1 # Show last time frame
)

## 5. Further Considerations
To make this notebook fully functional for Infrared Thermography:
1. **Implement `InfraredThermographyOperator` accurately**: 
    - This requires solving the heat equation (possibly with material properties like thermal conductivity, diffusivity, density) for a given subsurface heat source/defect distribution and boundary conditions. Finite Difference Method (FDM) or Finite Element Method (FEM) are common.
    - The adjoint operator (`op_adj`) must be the correct mathematical adjoint of the implemented forward heat model. This is crucial for gradient-based reconstruction algorithms to converge correctly.
2. **Model Specific IRT Techniques**: 
    - *Pulsed Thermography*: Model the transient thermal response after a short heat pulse.
    - *Lock-in Thermography*: Model the steady-state periodic response to a modulated heat source.
3. **Calibration**: Relate pixel values in the IR camera images to actual temperature values. Consider emissivity corrections.
4. **Noise Model**: Add realistic noise to the simulated surface temperature data (e.g., thermal noise, detector noise).
5. **Quantitative Inversion**: For estimating actual defect parameters (size, depth, thermal resistance) rather than just a qualitative map, the forward model must be very accurate, and the inversion might require more sophisticated regularization or optimization.